In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    "udp_wcm_bronze_cx_loyalty",
    "lyt_customer_merchant_rels",
    proc_date,
    "temp_cx_loyalty_lyt_customer_merchant_rels"
)

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_customer_merchant_rels
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT *
FROM temp_cx_loyalty_lyt_customer_merchant_rels
WHERE lower(is_deleted) = 'false'
QUALIFY ROW_NUMBER() OVER (PARTITION BY merchant_id, merchant_customer_id ORDER BY update_date DESC) = 1
""")